In [ ]:
#Question 6
# The ipynb files had to be modified based on the transformers used in custom transformers
# We can see that the f1 scores are more or less similar to that of the WEKA results
# The same goes for the Logistic Regression model. Although I don't have access to the inner workings of the
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

data = pd.read_csv('/content/UniversalBank.csv')

data.drop(columns=['ID', 'ZIP Code'], errors='ignore', inplace=True)

data = pd.get_dummies(data, columns=['Education'], drop_first=True)

y = data['Personal Loan']
X = data.drop(columns=['Personal Loan'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

grid_search_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)

best_lr = grid_search_lr.best_estimator_
y_pred_lr = best_lr.predict(X_test)
print("Best Logistic Regression Model:", grid_search_lr.best_params_)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

param_grid_dt = {
    'max_depth': [3, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

grid_search_dt = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_dt.fit(X_train, y_train)

best_dt = grid_search_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)
print("Best Decision Tree Model:", grid_search_dt.best_params_)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
print("Best Random Forest Model:", grid_search_rf.best_params_)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Best Logistic Regression Model: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Logistic Regression Accuracy: 0.967
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       895
           1       0.91      0.76      0.83       105

    accuracy                           0.97      1000
   macro avg       0.94      0.88      0.91      1000
weighted avg       0.97      0.97      0.97      1000

Best Decision Tree Model: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Decision Tree Accuracy: 0.991
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       895
           1       0.98      0.93      0.96       105

    accuracy                           0.99      1000
   macro avg       0.99      0.97      0.98      1000
weighted avg       0.99      0.99      0.99      1000

Best Random Forest Model: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 2

##Question 7


Now the problem with this is that the pipeline was only dealing with one dataset. Here, we already have the split doen for us. This means that we can just use preprocessing as is and get the fields adjusted based on the attributes for the sonar data now.

KNN: Performance depends on the choice of k. If the dataset is noisy or has similar class distributions, KNN may misclassify.

Naïve Bayes: Works well if features our are independent, but the assumption for the assignment is that it might not hold in Sonar data.

SVM: Works well for high-dimensional data like this one but we know that it  requires tuning. If the classes sonar has have non linear relationships and are not linearly separable, kernel selection becomes more important with SVM.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


In [ ]:

# Custom transformers was the main issue
# you will need to
#as the exception, I have used the pipeline from sklearn in case this does not work.
try:
    from custom_transformers import DataFrameSelector, ZeroVariance, FindCorrelation
    from custom_transformers import OptionalStandardScaler, ManualDropper, PipelineChecker
except ImportError:
    print("Custom transformers not found.")

train_data = pd.read_csv("/content/sonar_train.csv")
test_data = pd.read_csv("/content/sonar_test.csv")
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
base_pipeline = Pipeline([
    ('scaler', StandardScaler())
])
knn_pipeline = Pipeline([
    ('preprocess', base_pipeline),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

nb_pipeline = Pipeline([
    ('preprocess', base_pipeline),
    ('nb', GaussianNB())
])

svm_pipeline = Pipeline([
    ('preprocess', base_pipeline),
    ('svm', SVC(kernel='linear', C=1.0))
])


knn_pipeline.fit(X_train, y_train)
nb_pipeline.fit(X_train, y_train)
svm_pipeline.fit(X_train, y_train)
knn_pred = knn_pipeline.predict(X_test)
nb_pred = nb_pipeline.predict(X_test)
svm_pred = svm_pipeline.predict(X_test)
knn_acc = accuracy_score(y_test, knn_pred)
nb_acc = accuracy_score(y_test, nb_pred)
svm_acc = accuracy_score(y_test, svm_pred)

print(f"KNN Accuracy: {knn_acc:.4f}")
print(f"Naive Bayes Accuracy: {nb_acc:.4f}")
print(f"SVM Accuracy: {svm_acc:.4f}")
